# YOLO PREDICTIONS

## Install Necessary Packages 

In [1]:
pip install opencv-python==4.6.0.66

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install PyYAML

## Import Necessary Libraries

In [3]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

## STEP 1 : LOAD YAML FILE

In [4]:
with open('data.yaml', mode = 'r') as f:
    data_yaml = yaml.load(f, Loader = SafeLoader)
    
labels = data_yaml['names']
print(labels)

['person', 'car', 'chair', 'bottle', 'pottedplant', 'bird', 'dog', 'sofa', 'bicycle', 'horse', 'boat', 'motorbike', 'cat', 'tvmonitor', 'cow', 'sheep', 'aeroplane', 'train', 'diningtable', 'bus']


## STEP2 : LOAD YOLO MODEL with OPENCV

In [5]:
yolo = cv2.dnn.readNetFromONNX('./Model5/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

## STEP 3: Load the image

In [6]:
img = cv2.imread('./street_image.jpg')
image = img.copy()

In [7]:
# To View the image

'''
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\ncv2.imshow('image',image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

<img src = 'street_image.jpg'>

## STEP 4 : Retune the Image

In [8]:
# Check Height Width and Depth of image

row, col, d = image.shape
row, col, d

(956, 1920, 3)

### TASK : Covert our image into Square Image

In [9]:
# 1. decide the larger between row, col
max_rc = max(row, col)

# 2. Black Shell Matrix
input_image = np.zeros((max_rc, max_rc, 3), dtype = np.uint8)

In [10]:
# Check the Black Square Matrix

'''
cv2.imshow('input_image',input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\ncv2.imshow('input_image',input_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [11]:
# 3. Fill the Black Matrix with our Image

input_image [0:row, 0:col] = image

In [12]:
# Check the Converted Image

'''
cv2.imshow('input_image',input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\ncv2.imshow('input_image',input_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [13]:
## STEP 4 : GET PREDICTIONS FROM SQUARE ARRAY

INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image , 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop= False)
yolo.setInput(blob)

# Prediction from YOLO
preds = yolo.forward() 

In [14]:
print(preds)

[[[6.9053793e+00 5.4312000e+00 1.3816580e+01 ... 1.9329831e-02
   7.8076990e-03 6.3514709e-03]
  [1.3570687e+01 5.3768034e+00 2.5515362e+01 ... 2.0302750e-02
   7.6320907e-03 4.5632645e-03]
  [1.7675758e+01 5.5848465e+00 2.9498156e+01 ... 2.6307669e-02
   9.7375531e-03 8.5721789e-03]
  ...
  [5.6134430e+02 5.9979688e+02 1.5506219e+02 ... 3.8080622e-02
   1.6806498e-02 2.0614417e-02]
  [5.8220282e+02 6.0063727e+02 1.1114612e+02 ... 4.4948500e-02
   2.4509836e-02 2.3215117e-02]
  [6.0970844e+02 6.0693042e+02 1.1702031e+02 ... 6.6256702e-02
   2.9338455e-02 3.4287363e-02]]]


In [15]:
preds.shape

(1, 25200, 25)

In [16]:
# Conclusion : 

'''
No of Bounding Boxes Detected by Model = 25200

for each and every Bounding Box those info are available in 25 coloumns

'''

'\nNo of Bounding Boxes Detected by Model = 25200\n\nfor each and every Bounding Box those info are available in 25 coloumns\n\n'

SAMPLE YOLO OUTPUT

<img src = 'Supporting Image\Yolo Structure.png'>

## STEP 5:  NON MAX SUPPRESSION 

### Filter Detection based on Confidence 0.4 and Probability Score 0.25

In [17]:
detections = preds[0]

boxes = []
confidences = []
classes = []

In [18]:
# Caclculate Width and Height of the Image

image_w, image_h = input_image.shape[:2]
image_w, image_h

(1920, 1920)

In [19]:
# Calculate X_Factor & Y_Factor

x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

x_factor, y_factor

(3.0, 3.0)

In [20]:
# Filter Detection based on Confidence 0.4 and Probability Score 0.25

for i in range(len(detections)):
    row = detections[i]
    
    # Confidence of detecting an Object
    confidence = row[4] 
    
    if confidence > 0.4: 
        
        # Maximum Probability from 20 Objects
        class_score = row[5:].max()
        
        # Which Class gives Max Probablitiy
        class_id = row[5:].argmax()
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            
            # CONSTRUCT BOUNDING BOXES from the values
            # Left, TOP, WIDTH & HEIGHT
            
            left   = int((cx - 0.5*w)* x_factor)
            top    = int((cy - 0.5*h)* y_factor)
            width  = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left, top, width, height])
            
            # Append Values into List
            confidences.append (confidence)
            boxes.append (box)
            classes.append (class_id)
       

### Clean Duplicate Values & Store it in Numpy List 

In [21]:
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

### Non Max Suppression Operation

In [22]:
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

In [23]:
len(index)

27

### Draw the BOUNDING BOX

In [24]:
for ind in index : 
    
    # Extract Bounding Box
    x, y, w, h = boxes_np[ind]
    bb_conf = confidences_np[ind] * 100
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name} : {bb_conf}%'
    
    cv2.rectangle(image, (x,y), (x+w, y+h), (0,255, 0), 2)
    cv2.rectangle(image, (x,y-30), (x+w, y), (255,255, 255), -1)
    
    cv2.putText(image, text, (x,y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0),1)

In [ ]:
cv2.imshow('Original',img)
cv2.imshow('Predicted',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Original Image - Rehaped

<img src = 'Supporting Image\Original Image.jpeg'>

#### Predicted Image with Bounding Boxes

<img src = 'Supporting Image\Predicted Image.jpeg'>

## THE END